In [1]:
import pyprind
import pandas as pd
from string import punctuation
import re
import numpy as np

In [2]:
df = pd.read_csv('movie_data.csv',encoding='utf-8')

In [3]:
from collections import Counter
c = Counter()
c.update('abc')

In [4]:
c

Counter({'a': 1, 'b': 1, 'c': 1})

In [5]:
c['a']

1

In [6]:
c.update(['a','b'])

In [7]:
c

Counter({'a': 2, 'b': 2, 'c': 1})

In [8]:
c.update({'c':3})

In [9]:
c

Counter({'a': 2, 'b': 2, 'c': 4})

In [12]:
c.most_common(1)#가장 최빈 한 개 출력

[('c', 4)]

In [13]:
c.most_common()

[('c', 4), ('a', 2), ('b', 2)]

In [14]:
c.most_common(2)#최빈값 2개 출력

[('c', 4), ('a', 2)]